In [1]:
import itertools, json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import colors

import sys
sys.path.append('..')
from lib import *
from lib.maxent import *

In [2]:
output = True
N = 9
q = naminoacids

proteome = proteome_path('Human')
seed = 1234
prng = np.random.RandomState(seed)

In [4]:
params = np.load('data/Human_full_k9.npz')
hi = params['hi']
Jij = params['Jij']

In [3]:
datasets = ['train', 'test', 'model']
sample_matrices = {}
for dataset in datasets:
    sample_matrices[dataset] =  np.loadtxt('data/%s_matrix.csv.gz' % dataset).astype(int)

In [40]:
energies = [energy_potts(x, hi, Jij) for x in sample_matrices['model']]

In [55]:
F0 = -np.sum(np.log(np.sum(np.exp(hi), axis=1)))

In [96]:
Fprime1 = np.mean([energy_potts(x, np.zeros_like(hi), Jij) for x in sample_matrices['model']])

In [148]:
def Fprime(alpha):
    jump = lambda x: local_jump(x, q)
    x0 = prng.randint(q, size=N)
    matrix = mcmcsampler(x0, lambda x: energy_potts(x, hi, alpha*Jij), jump, 1e6, nsample=10, nburnin=1e3)
    return np.mean([energy_potts(x, np.zeros_like(hi), Jij) for x in matrix])

In [149]:
Fprime0 = Fprime(0.0)

In [150]:
Fprime05 = Fprime(0.5)

In [117]:
Fint = scipy.integrate.simps([Fprime0, Fprime05, Fprime1], dx=0.5)
Fint, 0.5*(Fprime1+Fprime0)

(0.11028362838063799, 0.09120436854995548)

In [112]:
F0, np.mean(energies), 0.5*(Fprime1+Fprime0), Fprime05

(-28.01652616764632,
 -2.072497153551555,
 0.09120436854995548,
 0.11982325829597922)

In [80]:
energies_ind = [energy_potts(x, hi, np.zeros_like(Jij)) for x in independent_matrix]

In [81]:
np.mean(energies_ind), np.mean(energies)

(-1.9631214994488029, -2.072497153551555)

In [123]:
def calc_Sind(hi):
    fis = np.exp(hi)/np.sum(np.exp(hi), axis=1)[:, np.newaxis]
    return np.sum(scipy.stats.entropy(fis.T))

In [124]:
Sind = calc_Sind(hi)
S = np.mean(energies) - (F0 + Fint)

In [140]:
Suni = np.log2(20)

In [142]:
Suni - Sind*np.log2(np.exp(1))/N, Suni - S*np.log2(np.exp(1))/N

(0.14563129029025212, 0.1807929442698173)

In [127]:
Sind, np.mean(energies_ind) - F0

(26.05309519759196, 26.053404668197516)

In [130]:
df = pd.read_csv('../kmerentropy/data/entropy.csv')

In [137]:
np.array(df['Human'])/np.arange(1, 6)

array([4.17756346, 4.16934288, 4.16129658, 4.14931991, 4.11369582])

In [147]:
(Sind-S)*np.log2(np.exp(1))

0.3164548858160828